In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!mkdir -p ~/.kaggle



import json 

token = {"username":"seunghwan1228","key":"2a8fb35ecd319bf10a02713c306059ca"}

with open('/content/drive/My Drive/Colab Notebooks/Kaggle Kernel/kaggle.json', 'w') as file:
  json.dump(token, file)
  
  
  
!cp drive/'My Drive'/'Colab Notebooks'/'Kaggle Kernel'/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json


# down load path

!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

kaggle.json
total 4
-rw------- 1 root root 72 Nov  2 05:16 kaggle.json
{"username": "seunghwan1228", "key": "2a8fb35ecd319bf10a02713c306059ca"}Warning: Looks like you're using an outdated API Version, please consider updating (server 1.5.6 / client 1.5.4)
 94% 257M/273M [00:03<00:00, 82.3MB/s]
100% 273M/273M [00:03<00:00, 75.4MB/s]
 75% 9.00M/12.0M [00:00<00:00, 29.3MB/s]
100% 12.0M/12.0M [00:00<00:00, 34.5MB/s]
  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 74.1MB/s]


In [3]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import numpy as np

In [0]:
df = pd.read_csv('/content/train.csv')

In [5]:
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [6]:
df.shape

(1804874, 45)

In [0]:
text_data = df['comment_text']
label_data = df['target']

In [0]:
def label_binary(x):
  if x >= 0.5:
    return 1.
  else :
    return 0.

In [0]:
label_data= label_data.apply(label_binary)

In [0]:
x_train, x_val, y_train, y_val = train_test_split(text_data, label_data, train_size=1000000, stratify = label_data, random_state=1228)

In [11]:
print(x_train.shape)
print(y_train.shape)

(1000000,)
(1000000,)


In [0]:
text_data = x_train
label_data = y_train

In [13]:
!pip install bert-tensorflow

In [14]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
x_train, x_val, y_train, y_val = train_test_split(text_data, label_data, test_size=0.2, random_state=1228, stratify=label_data)

In [16]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(800000,)
(200000,)
(800000,)
(200000,)


In [19]:
x_train.head()

893226     What's he want -- a 20 foot statue of himself ...
1731853    Light weight analysis ... light weight conclus...
1396609    Dano, no doubt this was an important draft, bu...
987276     Please don't fall asleep as I am anxiously awa...
1497311    Congressman Tim Ryan. \nI like where you're go...
Name: comment_text, dtype: object

In [0]:
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_val, y_val], axis=1)

In [18]:
train.head()

,comment_text,target
893226,What's he want -- a 20 foot statue of himself ...,0.0
1731853,Light weight analysis ... light weight conclus...,0.0
1396609,"Dano, no doubt this was an important draft, bu...",0.0
987276,Please don't fall asleep as I am anxiously awa...,0.0
1497311,Congressman Tim Ryan. \nI like where you're go...,0.0


In [19]:
train.columns

Index(['comment_text', 'target'], dtype='object')

In [0]:
DATA_COLUMN = 'comment_text'
LABEL_COLUMN= 'target'
label_list = [0, 1]

In [0]:
train_InputeExample = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                             text_a = x[DATA_COLUMN],
                                                                             text_b = None,
                                                                             label = x[LABEL_COLUMN]), axis= 1)

test_InputeExample = test.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                           text_a = x[DATA_COLUMN],
                                                                           text_b = None,
                                                                           label = x[LABEL_COLUMN]), axis= 1)

In [0]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run(
          [
              tokenization_info["vocab_file"],
              tokenization_info["do_lower_case"],
          ]
      )
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)


In [0]:
module = hub.Module(BERT_MODEL_HUB)

In [24]:
module(signature="tokenization_info", as_dict=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


{'do_lower_case': <tf.Tensor 'module_apply_tokenization_info/Const:0' shape=() dtype=bool>,
 'vocab_file': <tf.Tensor 'module_apply_tokenization_info/vocab_file:0' shape=() dtype=string>}

In [25]:
tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [26]:
tokenizer.tokenize('hello, we are going to make bert NLP model')

['hello',
 ',',
 'we',
 'are',
 'going',
 'to',
 'make',
 'bert',
 'nl',
 '##p',
 'model']

In [0]:
def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [27]:
# max_seq_length = 256

MAX_LEN = 256

(train_input_ids, train_input_masks, train_segment_ids, train_labels) = bert.run_classifier.convert_examples_to_features(train_InputeExample, label_list, MAX_LEN, tokenizer)
(test_input_ids, test_input_masks, test_segment_ids, test_labels) = bert.run_classifier.convert_examples_to_features(test_InputeExample, label_list, MAX_LEN, tokenizer)

# (train_input_ids, train_input_masks, train_segment_ids, train_labels) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)

INFO:tensorflow:Writing example 0 of 800000


INFO:tensorflow:Writing example 0 of 800000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Writing example 780000 of 800000


INFO:tensorflow:Writing example 780000 of 800000


INFO:tensorflow:Writing example 790000 of 800000


INFO:tensorflow:Writing example 790000 of 800000


ValueError: ignored

In [36]:
train_example[0]

# Create Model

In [32]:
MAX_LEN

256

In [0]:
class BertLayer(tf.layers.Layer):
    def __init__(self, n_fine_tune_layers=10, **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            bert_path,
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )
        trainable_vars = self.bert.variables
        
        # Remove unused layers
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
        
        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]
        
        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)
        
        # Add non-trainable weights
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)
        
        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
            "pooled_output"
        ]
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)